# Part V: Finding property deals

This part is a concatenation of all previous files.
<br><br>
As a result of the operation of the notebook:
1. A certain number of offers are scrapped
2. offer data is transformed into a form that enables predictions
3. missing BUILD_YEAR are completed if possible
4. The model determines the price
5. a summary of offers is returned, with information on the extent to which the price is profitable

In [1]:
import numpy as np
import csv
import pandas as pd
from typing import List
from modules.otodom_scraper import get_subpages_list, get_offers, scrape_offers
from modules.property_deal import preprocess_new_offers, fill_build_years, predict_prices_per_m2
pd.set_option('display.max_columns', None)
import joblib
import pickle
import warnings
warnings.filterwarnings('ignore')

### Settings

In [3]:
# provide a template of url. It should end with &page= to generate correct links
template_url = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/wroclaw?distanceRadius=0&page='

# set number of subpages to scrap (its 36 offers per subpage)
number_of_subpages = 1

### Scrapping new offers

In [4]:
subpages_list = get_subpages_list(template_url, number_of_subpages)
offers_list = get_offers(subpages_list)
df_new_offers = scrape_offers(offers_list)

Added offers from subpage: 30 / 36 ---> 83.33% : wykonczone-mieszkanie-blisko-zieleni-ID4jPjR to offers list 
Added offers from subpage: 31 / 36 ---> 86.11% : 5-pokojowe-mieszkanie-103m2-2-tarasy-ID4i3j7 to offers list 
Added offers from subpage: 32 / 36 ---> 88.89% : mieszkanie-72-m2-horbaczewskiego-ID4kKMp to offers list 
Added offers from subpage: 33 / 36 ---> 91.67% : 4-pokojowe-mieszkanie-77m2-balkon-bezposrednio-ID4cT20 to offers list 
Added offers from subpage: 34 / 36 ---> 94.44% : 4-pokojowe-mieszkanie-105m2-ogrodek-bez-prowizji-ID4jMhv to offers list 
Added offers from subpage: 35 / 36 ---> 97.22% : 1-pokojowe-mieszkanie-25m2-bez-prowizji-ID4hgQW to offers list 
Added offers from subpage: 36 / 36 ---> 100.00% : 3-pokojowe-mieszkanie-62m2-2-balkony-ID4cT1Z to offers list 
------------------------------
Successfully created DataFrame
Rows: 36
Columns: 31


### Preprocessing

In [5]:
df_new_offers.drop_duplicates(subset='SLUG', keep='first', inplace=True)
df_new_offers_preprocessed = preprocess_new_offers(df_new_offers)

print(f'Number of offers left after preprocessing: {df_new_offers_preprocessed.shape[0]}')

Number of offers left after preprocessing: 34


### Receive missing build years


In [6]:
df_with_preds = fill_build_years(df_new_offers_preprocessed)

Number of missing year of build: 3
Number of received year of build: 2


### Predict price per m2

In [7]:
df_with_preds = predict_prices_per_m2(df_with_preds)

Number of loaded offers: 33


### Combine DataFrames into main file

In [8]:
# build main dataframe basing on the original, human friendly data
df_main = df_new_offers.drop(columns=['ID', 'TRANSACTION_TYPE', 'ESTATE_CATEGORY', 'CITY', 'SUBDISTRICT', 'PROVINCE','COUNTRY', 'LATITUDE', 'LONGITUDE', 'SUBREGION']).copy()

# fill missing build years
df_main['BUILD_YEAR'] = df_main['BUILD_YEAR'].fillna(df_main['SLUG'].map(df_with_preds.set_index('SLUG')['BUILD_YEAR']))

# fill missings as unknown
df_main = df_main.fillna('unknown')

# add predictions
df_main = pd.merge(df_main, df_with_preds[['PREDICTED_PRICE_PER_M2', 'SLUG']], on='SLUG', how='left')
df_main['PREDICTED_PRICE_PER_M2'] = df_main['PREDICTED_PRICE_PER_M2'].round(2)
df_main = df_main.dropna()

# add additional calculations
df_main['PRICE_PER_M2'] = df_main['PRICE'].astype(float) / df_main['AREA'].astype(float) 
df_main['PRICE_PER_M2'] = df_main['PRICE_PER_M2'].round(2)
df_main['PREDICTED_PRICE'] = df_main['AREA'].astype(float) * df_main['PREDICTED_PRICE_PER_M2']
df_main['PREDICTED_PRICE'] = df_main['PREDICTED_PRICE'].astype(int)
df_main = df_main[df_main['PREDICTED_PRICE_PER_M2'] <= 13_500]
df_main = df_main[df_main['PRICE_PER_M2'] <= 13_500]
df_main['AREA'] = df_main['AREA'].astype(float)
# change order of columns
for col in [ 'PREDICTED_PRICE',   'PRICE',  'PREDICTED_PRICE_PER_M2'  ,'PRICE_PER_M2','SLUG']:

    slug_col = df_main.pop(col)
    df_main.insert(0, col, slug_col)

### Add calculations

In [9]:
df_main['DIFF_PRICE'] = df_main['PREDICTED_PRICE'].astype(int) - df_main['PRICE'].astype(int) 
df_main['DIFF_PRICE_PER_M2'] =  df_main['PREDICTED_PRICE_PER_M2'].astype(float)- df_main['PRICE_PER_M2'].astype(float) 

df_main.insert(5, 'DIFF_PRICE', df_main.pop('DIFF_PRICE'))
df_main.insert(5, 'DIFF_PRICE_PER_M2', df_main.pop('DIFF_PRICE_PER_M2'))

df_main = df_main.sort_values('DIFF_PRICE', ascending = 0)

# format numbers
columns_to_format = ['PRICE_PER_M2', 'PREDICTED_PRICE_PER_M2', 'PRICE', 'PREDICTED_PRICE', 'DIFF_PRICE_PER_M2', 'DIFF_PRICE'  ]

format_column = lambda x: '{:,.2f}'.format(float(x))


for col in columns_to_format:
    df_main[col] = df_main[col].apply(format_column)

DataFrame compares the sales prices with the prices predicted by the model. The greater the difference, the greater the potential price opportunity  the offer may be.

In [10]:
df_main.head(10)

,SLUG,PRICE_PER_M2,PREDICTED_PRICE_PER_M2,PRICE,PREDICTED_PRICE,DIFF_PRICE_PER_M2,DIFF_PRICE,MARKET_TYPE,USER_TYPE,AREA,ROOMS_NUM,BUILD_YEAR,FLOOR_NO,BUILDING_FLOORS_NUM,BUILDING_MATERIAL,BUILDING_OWNERSHIP,BUILDING_TYPE,CONSTRUCTION_STATUS,EXTRAS_TYPES,EQUIPMENT_TYPES,WINDOWS_TYPE,RENT,HEATING,SECURITY_TYPES,DISTRICT,STREET
17,78-64m-4-pokoje-kuchnia-duze-mieszkanie-piwnic...,"7,248.22","9,647.52","570,000.00","758,680.00","2,399.30","188,680.00",secondary,agency,78.64,4,1967,floor_6,10,unknown,full_ownership,block,to_renovation,"balcony, basement, lift, separate_kitchen",unknown,plastic,700,urban,unknown,unknown,unknown
2,gora-domu-z-ogrodem-w-lesnicy-ID4kbnp,"6,567.59","8,345.34","685,000.00","870,418.00","1,777.75","185,418.00",secondary,agency,104.30,4,1930,floor_1,2,brick,full_ownership,house,ready_to_use,"garage, garden",unknown,plastic,unknown,gas,anti_burglary_door,Leśnica,ul. Trzmielowicka
4,inwestycyjne-3-pokoje-centrum-miasta-blisko-ue...,"9,126.82","10,892.09","439,000.00","523,909.00","1,765.27","84,909.00",secondary,agency,48.10,3,1970,floor_7,10,unknown,full_ownership,block,unknown,"basement, lift",unknown,plastic,500,urban,entryphone,Krzyki,ul. Drukarska
0,4-pok-aneks-krzyki-komunikacja-mpk-iiiq-2023-I...,"7,849.98","8,587.05","579,250.00","633,638.00",737.07,"54,388.00",primary,agency,73.79,4,2023,floor_1,3,unknown,full_ownership,block,to_completion,"balcony, basement, garage, lift, usable_room",unknown,plastic,unknown,urban,"anti_burglary_door, entryphone, monitoring",Krzyki,unknown
16,3-pokoje-61-25m2-z-tarasem-jagodno-ID4kRMu,"10,775.51","11,452.88","660,000.00","701,488.00",677.37,"41,488.00",secondary,private,61.25,3,2021,floor_1,4,brick,full_ownership,block,ready_to_use,"basement, garage, lift, terrace, usable_room","dishwasher, fridge, furniture, oven, stove, tv...",plastic,496,urban,"anti_burglary_door, entryphone, roller_shutters",Jagodno,Kajdasza
19,okolice-gubinskiej-piwnica-ID4kUNJ,"11,364.38","12,679.59","349,000.00","389,390.00","1,315.21","40,390.00",secondary,agency,30.71,1,1995,floor_4,5,unknown,unknown,block,ready_to_use,basement,"fridge, furniture, washing_machine",plastic,460,urban,entryphone,Nowy Dwór,unknown
15,4-pokoje-taras-23m2-2-miejsca-parkingowe-ID4jJ3M,"11,744.36","12,126.05","849,000.00","876,592.00",381.69,"27,592.00",secondary,agency,72.29,4,2019,ground_floor,3,silikat,full_ownership,block,ready_to_use,"air_conditioning, balcony, basement, garage, l...","dishwasher, fridge, furniture, oven, stove, tv...",plastic,800,urban,"anti_burglary_door, entryphone, monitoring, ro...",Psie Pole,ul. Ferdynanda Magellana
3,gotowe-4-pokoje-na-starym-miescie-ID4kK4R,"11,179.40","11,486.74","821,462.00","844,045.00",307.34,"22,583.00",primary,agency,73.48,4,2022,ground_floor,7,other,full_ownership,apartment,to_completion,"balcony, garage, lift",unknown,plastic,590,urban,"anti_burglary_door, entryphone, monitoring, ro...",Szczepin,ul. Gnieźnieńska
25,centrum-59-44-m2-tramwaj-2-pokoje-ID4jM2O,"8,832.44","9,159.71","525,000.00","544,453.00",327.27,"19,453.00",secondary,agency,59.44,2,1995,floor_5,5,unknown,unknown,block,unknown,"balcony, basement",unknown,wooden,500,urban,entryphone,Stare Miasto,ul. Legnicka
23,rodzinne-3-pokoje-na-stablowicach-ID4jQp2,"9,087.78","9,331.51","528,000.00","542,160.00",243.73,"14,160.00",secondary,agency,58.10,3,2017,floor_1,3,other,full_ownership,block,unknown,"balcony, lift",unknown,plastic,392,boiler_room,entryphone,Stabłowice,ul. Arbuzowa
